# Example: Story-Driven Narrative Visualizations

This notebook demonstrates creating compelling, narrative-driven visualizations for communicating land clearing stories.

In [ ]:
# Import required modules
from aus_land_clearing.data import load_dea_fractional_cover, load_slats_data
from aus_land_clearing.processing import (
    extract_time_series,
    calculate_change_statistics,
    calculate_vegetation_indices,
    aggregate_by_period
)
from aus_land_clearing.visualization import (
    create_time_series_plot,
    create_animation,
    create_narrative_visualization
)
from aus_land_clearing.utils import get_study_area_bbox, get_time_range
import matplotlib.pyplot as plt
import numpy as np

## Story 1: Agricultural Expansion in Queensland

This narrative focuses on agricultural expansion and land clearing for farming in Queensland's interior regions.

In [ ]:
# Load data for Queensland
bbox_qld = get_study_area_bbox('queensland')
start_year, end_year = get_time_range()
time_range = (f'{start_year}-01-01', f'{end_year}-12-31')

# Load fractional cover data
fc_qld = load_dea_fractional_cover(bbox_qld, time_range, bands=['PV', 'NPV', 'BS'])

# Calculate vegetation indices
fc_indices = calculate_vegetation_indices(fc_qld)

# Calculate change statistics
ag_expansion_stats = calculate_change_statistics(
    fc_indices,
    variable='total_vegetation',
    baseline_period=('1988-01-01', '1995-12-31'),
    comparison_period=('2010-01-01', '2020-12-31')
)

print("Agricultural Expansion Impact:")
print(f"Change in total vegetation: {ag_expansion_stats.get('percent_change', 'N/A'):.1f}%")
print(f"This represents approximately {abs(ag_expansion_stats.get('absolute_change', 0)):.0f} units of vegetation loss")

In [ ]:
# Create narrative visualization
create_narrative_visualization(
    fc_indices,
    variable='total_vegetation',
    narrative_type='before_after',
    output_path='../data/outputs/story1_agricultural_expansion.png',
    figsize=(16, 8)
)

print("Narrative visualization saved!")

## Story 2: Urban Development in Coastal NSW

This narrative examines urban expansion along the NSW coast and its impact on vegetation cover.

In [ ]:
# Load data for NSW coastal region
bbox_nsw = get_study_area_bbox('new_south_wales')
fc_nsw = load_dea_fractional_cover(bbox_nsw, time_range, bands=['PV'])

# Aggregate to annual
annual_nsw = aggregate_by_period(fc_nsw, variable='PV', period='year')

# Extract time series
pv_ts_nsw = extract_time_series(annual_nsw, variable='PV', method='mean')

# Create multi-temporal view
create_narrative_visualization(
    annual_nsw,
    variable='PV',
    narrative_type='multi_temporal',
    output_path='../data/outputs/story2_urban_development.png',
    figsize=(18, 12)
)

print("Urban development narrative saved!")

## Story 3: Vegetation Regeneration and Recovery

This narrative highlights areas of vegetation regeneration and recovery, showing positive environmental trends.

In [ ]:
# Calculate vegetation health trends
health_stats = calculate_change_statistics(
    fc_indices,
    variable='vegetation_health',
    baseline_period=('2000-01-01', '2005-12-31'),
    comparison_period=('2015-01-01', '2020-12-31')
)

print("Vegetation Recovery:")
print(f"Change in vegetation health: {health_stats.get('percent_change', 'N/A'):.1f}%")

# Create change map
create_narrative_visualization(
    fc_indices,
    variable='vegetation_health',
    narrative_type='change_map',
    output_path='../data/outputs/story3_vegetation_recovery.png',
    figsize=(14, 10)
)

print("Recovery narrative saved!")

## Story 4: Climate and Fire Impacts

This narrative examines the impact of drought and fire on vegetation cover over time.

In [ ]:
# Focus on periods with known climate impacts
# Millennium Drought: 2001-2009
# Black Summer Fires: 2019-2020

drought_impact = calculate_change_statistics(
    fc_qld,
    variable='PV',
    baseline_period=('1990-01-01', '2000-12-31'),
    comparison_period=('2001-01-01', '2009-12-31')
)

fire_impact = calculate_change_statistics(
    fc_nsw,
    variable='PV',
    baseline_period=('2015-01-01', '2018-12-31'),
    comparison_period=('2019-01-01', '2020-12-31')
)

print("Climate Impact Analysis:")
print(f"Drought impact (2001-2009): {drought_impact.get('percent_change', 'N/A'):.1f}%")
print(f"Fire impact (2019-2020): {fire_impact.get('percent_change', 'N/A'):.1f}%")

In [ ]:
# Create timeline visualization
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Queensland drought timeline
if not pv_ts_nsw.empty and 'PV' in pv_ts_nsw.columns:
    axes[0].plot(pv_ts_nsw.index, pv_ts_nsw['PV'], linewidth=2, color='darkgreen')
    axes[0].axvspan(2001, 2009, alpha=0.2, color='orange', label='Millennium Drought')
    axes[0].set_title('Queensland - Drought Impact on Vegetation', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('PV Cover (%)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

# NSW fire timeline
if not pv_ts_nsw.empty and 'PV' in pv_ts_nsw.columns:
    axes[1].plot(pv_ts_nsw.index, pv_ts_nsw['PV'], linewidth=2, color='darkgreen')
    axes[1].axvspan(2019, 2020, alpha=0.2, color='red', label='Black Summer Fires')
    axes[1].set_title('New South Wales - Fire Impact on Vegetation', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Year')
    axes[1].set_ylabel('PV Cover (%)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/outputs/story4_climate_fire_impacts.png', dpi=300, bbox_inches='tight')
plt.show()

print("Climate and fire impact narrative saved!")

## Creating an Integrated Narrative Dashboard

Combine multiple stories into a comprehensive dashboard.

In [ ]:
# Create comprehensive dashboard
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Title
fig.suptitle(
    'Eastern Australia Land Clearing: 1988-2024\nA Story of Change, Impact, and Recovery',
    fontsize=18,
    fontweight='bold',
    y=0.98
)

# Panel 1: Overall time series (top, spanning all columns)
ax1 = fig.add_subplot(gs[0, :])
ax1.set_title('Regional Vegetation Trends', fontsize=14, fontweight='bold')
ax1.set_ylabel('Vegetation Cover (%)')
ax1.set_xlabel('Year')
ax1.grid(True, alpha=0.3)

# Panel 2-4: Story snapshots (middle row)
ax2 = fig.add_subplot(gs[1, 0])
ax2.set_title('Agricultural\nExpansion', fontsize=12, fontweight='bold')

ax3 = fig.add_subplot(gs[1, 1])
ax3.set_title('Urban\nDevelopment', fontsize=12, fontweight='bold')

ax4 = fig.add_subplot(gs[1, 2])
ax4.set_title('Vegetation\nRecovery', fontsize=12, fontweight='bold')

# Panel 5-7: Impact analysis (bottom row)
ax5 = fig.add_subplot(gs[2, 0])
ax5.set_title('Drought Impact', fontsize=12, fontweight='bold')

ax6 = fig.add_subplot(gs[2, 1])
ax6.set_title('Fire Impact', fontsize=12, fontweight='bold')

ax7 = fig.add_subplot(gs[2, 2])
ax7.set_title('Net Change\n1988-2024', fontsize=12, fontweight='bold')

plt.savefig('../data/outputs/integrated_narrative_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()

print("Integrated narrative dashboard created!")

## Export for Presentations

Create high-quality exports suitable for reports and presentations.

In [ ]:
# Export settings for different use cases
export_settings = {
    'report': {'dpi': 300, 'format': 'png'},
    'presentation': {'dpi': 150, 'format': 'png'},
    'web': {'dpi': 96, 'format': 'png'},
    'print': {'dpi': 600, 'format': 'pdf'}
}

print("Export settings configured for multiple outputs")
print("Outputs saved to: ../data/outputs/")

## Key Takeaways

This notebook demonstrates:

1. **Multiple Narrative Perspectives**: Agricultural expansion, urban development, recovery, and climate impacts
2. **Temporal Context**: Using appropriate baseline and comparison periods
3. **Visual Storytelling**: Before/after, multi-temporal, and change maps
4. **Integrated Analysis**: Combining multiple stories into comprehensive dashboards
5. **Export Options**: Creating publication-ready outputs

## Next Steps

- Customize narratives for specific regions or time periods
- Add annotations and text overlays to enhance storytelling
- Combine with SLATS data for Queensland-specific analysis
- Create interactive visualizations using Plotly or similar tools
- Generate animations showing change over time